# TELL Analysis of 2012 Heat Wave

This notebook processes and makes plots of the temporal evolution of load during the course of the 2012 heat wave in the U.S.

In [ ]:
# Start by importing required packages and information about your operating system:
import os 
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from datetime import timedelta


## Set the Directory Structure

In [ ]:
# Identify the top-level data input and output directories:
tell_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave/data/tell_data/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave/plots'

# If the "image_output_dir" subdirectory doesn't exist then create it:
if not os.path.exists(image_output_dir):
   os.makedirs(image_output_dir)


## Set the Balancing Authority and Scenario You Want to Plot

In [ ]:
# Give the abbreviation for the BA you want to analyze:
ba_to_plot = 'PSCO'

# Set the scenario you want to analyze:
scenario = 'without_population_effects'


## Process the Load Data for Time Series Plotting

In [ ]:
# Read in the historical load data:
if scenario == 'with_population_effects': 
   hist_df = pd.read_csv((tell_data_input_dir + 'with_population_effects/' + 'BA_Loads_2012_With_Population_Effects.csv'))
   fut_df = pd.read_csv((tell_data_input_dir + 'with_population_effects/' + 'BA_Loads_2052_With_Population_Effects.csv'))
else:
   hist_df = pd.read_csv((tell_data_input_dir + 'without_population_effects/' + 'BA_Loads_2012_Without_Population_Effects.csv'))
   fut_df = pd.read_csv((tell_data_input_dir + 'without_population_effects/' + 'BA_Loads_2052_Without_Population_Effects.csv'))
    
hist_df = hist_df.loc[hist_df['BA'].isin([ba_to_plot])].copy()
hist_df['Year'] = pd.DatetimeIndex(hist_df['Time_UTC']).year
hist_df['Month'] = pd.DatetimeIndex(hist_df['Time_UTC']).month
hist_df['Day'] = pd.DatetimeIndex(hist_df['Time_UTC']).day
hist_df['Hour'] = pd.DatetimeIndex(hist_df['Time_UTC']).hour

fut_df = fut_df.loc[fut_df['BA'].isin([ba_to_plot])].copy()
fut_df['Year'] = pd.DatetimeIndex(fut_df['Time_UTC']).year
fut_df['Month'] = pd.DatetimeIndex(fut_df['Time_UTC']).month
fut_df['Day'] = pd.DatetimeIndex(fut_df['Time_UTC']).day
fut_df['Hour'] = pd.DatetimeIndex(fut_df['Time_UTC']).hour
    
plot_df = pd.merge(hist_df, fut_df, how='left', on=['BA', 'Month', 'Day', 'Hour'])
plot_df['date'] = plot_df[["Year_x","Month","Day","Hour"]].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
plot_df['Time_UTC'] = pd.to_datetime(plot_df['date'], format='%Y-%m-%d-%H')

# Compute the normalized loads:
plot_df['rcp45cooler_ssp3_norm'] = 100 * (plot_df['rcp45cooler_ssp3'].div(plot_df['historic']) - 1)
plot_df['rcp45cooler_ssp5_norm'] = 100 * (plot_df['rcp45cooler_ssp5'].div(plot_df['historic']) - 1)
plot_df['rcp45hotter_ssp3_norm'] = 100 * (plot_df['rcp45hotter_ssp3'].div(plot_df['historic']) - 1)
plot_df['rcp45hotter_ssp5_norm'] = 100 * (plot_df['rcp45hotter_ssp5'].div(plot_df['historic']) - 1)
plot_df['rcp85cooler_ssp3_norm'] = 100 * (plot_df['rcp85cooler_ssp3'].div(plot_df['historic']) - 1)
plot_df['rcp85cooler_ssp5_norm'] = 100 * (plot_df['rcp85cooler_ssp5'].div(plot_df['historic']) - 1)
plot_df['rcp85hotter_ssp3_norm'] = 100 * (plot_df['rcp85hotter_ssp3'].div(plot_df['historic']) - 1)
plot_df['rcp85hotter_ssp5_norm'] = 100 * (plot_df['rcp85hotter_ssp5'].div(plot_df['historic']) - 1)

# Only keep the columns that are needed:
plot_df = plot_df[['Time_UTC', 'historic', 
                   'rcp45cooler_ssp3', 'rcp45cooler_ssp3_norm', 
                   'rcp45cooler_ssp5', 'rcp45cooler_ssp5_norm',
                   'rcp45hotter_ssp3', 'rcp45hotter_ssp3_norm',
                   'rcp45hotter_ssp5', 'rcp45hotter_ssp5_norm',
                   'rcp85cooler_ssp3', 'rcp85cooler_ssp3_norm', 
                   'rcp85cooler_ssp5', 'rcp85cooler_ssp5_norm',
                   'rcp85hotter_ssp3', 'rcp85hotter_ssp3_norm',
                   'rcp85hotter_ssp5', 'rcp85hotter_ssp5_norm']].copy()

# Compute the date of the peak historical load:
peak_day = plot_df['Time_UTC'].loc[plot_df['historic'].idxmax()] 
min_date = pd.to_datetime(peak_day) - timedelta(days=3.5)
max_date = pd.to_datetime(peak_day) + timedelta(days=3.5)

# Compute the annual minimum and maximum loads:
min_load = 0.95*plot_df['historic'].min()
max_load = 1.05*plot_df['historic'].max()

# Return the dataframe:
plot_df


## Make the Time Series Plots

In [ ]:
# Make the plot:
plt.figure(figsize=(25, 20))
plt.rcParams['font.size'] = 16
plt.subplot(3,1,1)
plt.plot(plot_df['Time_UTC'], plot_df['historic'], color='k', linestyle='-', label='Historical', linewidth=2)
plt.plot([min_date, min_date], [min_load, max_load], color='g', linestyle='-', label='Peak Week', linewidth=3)
plt.plot([max_date, max_date], [min_load, max_load], color='g', linestyle='-', linewidth=3)
plt.ylim([min_load, max_load])
plt.xlim([datetime.date(2012, 1, 1), datetime.date(2012, 12, 31)])
plt.legend(loc='upper left')
plt.xlabel('')
plt.ylabel('Total Load [MW]')
plt.title(('Historical Total Loads in ' + ba_to_plot + ': 2012'))

plt.subplot(3,1,2)
plt.plot(plot_df['Time_UTC'], plot_df['historic'], color='k', linestyle='-', label='Historical', linewidth=2)
if scenario == 'with_population_effects':
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45cooler_ssp3'], color='b', linestyle='-', label='rcp45cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45cooler_ssp5'], color='b', linestyle=':', label='rcp45cooler_ssp5', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45hotter_ssp3'], color='cyan', linestyle='-', label='rcp45hotter_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45hotter_ssp5'], color='cyan', linestyle=':', label='rcp45hotter_ssp5', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85cooler_ssp3'], color='orange', linestyle='-', label='rcp85cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85cooler_ssp5'], color='orange', linestyle=':', label='rcp85cooler_ssp5', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85hotter_ssp3'], color='red', linestyle='-', label='rcp85hotter_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85hotter_ssp5'], color='red', linestyle=':', label='rcp85hotter_ssp5', linewidth=3)
else:
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45cooler_ssp3'], color='b', linestyle='-', label='rcp45cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45hotter_ssp3'], color='cyan', linestyle='-', label='rcp45hotter_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85cooler_ssp3'], color='orange', linestyle='-', label='rcp85cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85hotter_ssp3'], color='red', linestyle='-', label='rcp85hotter_ssp3', linewidth=3)
plt.xlim([min_date, max_date])
plt.legend(loc='upper left')
plt.xticks([])
plt.xlabel('')
plt.ylabel('Projected Total Load [MW]')
if scenario == 'with_population_effects':
   plt.title(('Absolute Climate and Socioeconomic Impacts in the Peak Load Week: 2052'))
else:
   plt.title(('Absolute Climate Only Impacts in the Peak Load Week: 2052'))

plt.subplot(3,1,3)
if scenario == 'with_population_effects':
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45cooler_ssp3_norm'], color='b', linestyle='-', label='rcp45cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45cooler_ssp5_norm'], color='b', linestyle=':', label='rcp45cooler_ssp5', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45hotter_ssp3_norm'], color='cyan', linestyle='-', label='rcp45hotter_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45hotter_ssp5_norm'], color='cyan', linestyle=':', label='rcp45hotter_ssp5', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85cooler_ssp3_norm'], color='orange', linestyle='-', label='rcp85cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85cooler_ssp5_norm'], color='orange', linestyle=':', label='rcp85cooler_ssp5', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85hotter_ssp3_norm'], color='red', linestyle='-', label='rcp85hotter_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85hotter_ssp5_norm'], color='red', linestyle=':', label='rcp85hotter_ssp5', linewidth=3)
else:
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45cooler_ssp3_norm'], color='b', linestyle='-', label='rcp45cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp45hotter_ssp3_norm'], color='cyan', linestyle='-', label='rcp45hotter_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85cooler_ssp3_norm'], color='orange', linestyle='-', label='rcp85cooler_ssp3', linewidth=3)
   plt.plot(plot_df['Time_UTC'], plot_df['rcp85hotter_ssp3_norm'], color='red', linestyle='-', label='rcp85hotter_ssp3', linewidth=3)
plt.plot([min_date, max_date], [0, 0], color='k', linestyle='-', linewidth=3)
plt.xlim([min_date, max_date])
plt.legend(loc='upper left')
plt.xticks([])
plt.xlabel('')
plt.ylabel('Load Change Relative to 2012 [%]')
if scenario == 'with_population_effects':
   plt.title(('Relative Climate and Socioeconomic Impacts in the Peak Load Week: 2052'))
else:
   plt.title(('Relative Climate Only Impacts in the Peak Load Week: 2052'))

if scenario == 'with_population_effects':
   filename = ('2012_heatwave_' + ba_to_plot + '_with_population_effects.png')
else:
   filename = ('2012_heatwave_' + ba_to_plot + '_without_population_effects.png')

plt.savefig(os.path.join(image_output_dir, filename), dpi=300, bbox_inches='tight', facecolor='white')


## Process the Load Data for BA Histogram Plots

In [ ]:
# Set the scenario you want to analyze:
scenario_to_analze = 'rcp85hotter_ssp5'


In [ ]:
# Read in the historical load data with population effects:
hist_with_df = pd.read_csv((tell_data_input_dir + 'with_population_effects/' + 'BA_Loads_2012_With_Population_Effects.csv'))
hist_with_df.rename(columns={'historic': 'historic_with_pop'}, inplace=True)
hist_with_df['Year'] = pd.DatetimeIndex(hist_with_df['Time_UTC']).year
hist_with_df['Month'] = pd.DatetimeIndex(hist_with_df['Time_UTC']).month
hist_with_df['Day'] = pd.DatetimeIndex(hist_with_df['Time_UTC']).day
hist_with_df['Hour'] = pd.DatetimeIndex(hist_with_df['Time_UTC']).hour

# Read in the projected load data with population effects:
fut_with_df = pd.read_csv((tell_data_input_dir + 'with_population_effects/' + 'BA_Loads_2052_With_Population_Effects.csv'))
fut_with_df = fut_with_df[['BA', 'Time_UTC', scenario_to_analze]].copy()
fut_with_df.rename(columns={scenario_to_analze: 'future_with_pop'}, inplace=True)
fut_with_df['Year'] = pd.DatetimeIndex(fut_with_df['Time_UTC']).year
fut_with_df['Month'] = pd.DatetimeIndex(fut_with_df['Time_UTC']).month
fut_with_df['Day'] = pd.DatetimeIndex(fut_with_df['Time_UTC']).day
fut_with_df['Hour'] = pd.DatetimeIndex(fut_with_df['Time_UTC']).hour
    
# Read in the historical load data without population effects:
hist_without_df = pd.read_csv((tell_data_input_dir + 'without_population_effects/' + 'BA_Loads_2012_Without_Population_Effects.csv'))
hist_without_df.rename(columns={'historic': 'historic_without_pop'}, inplace=True)
hist_without_df['Year'] = pd.DatetimeIndex(hist_without_df['Time_UTC']).year
hist_without_df['Month'] = pd.DatetimeIndex(hist_without_df['Time_UTC']).month
hist_without_df['Day'] = pd.DatetimeIndex(hist_without_df['Time_UTC']).day
hist_without_df['Hour'] = pd.DatetimeIndex(hist_without_df['Time_UTC']).hour

# Read in the projected load data without population effects:
fut_without_df = pd.read_csv((tell_data_input_dir + 'without_population_effects/' + 'BA_Loads_2052_Without_Population_Effects.csv'))
fut_without_df = fut_without_df[['BA', 'Time_UTC', scenario_to_analze]].copy()
fut_without_df.rename(columns={scenario_to_analze: 'future_without_pop'}, inplace=True)
fut_without_df['Year'] = pd.DatetimeIndex(fut_without_df['Time_UTC']).year
fut_without_df['Month'] = pd.DatetimeIndex(fut_without_df['Time_UTC']).month
fut_without_df['Day'] = pd.DatetimeIndex(fut_without_df['Time_UTC']).day
fut_without_df['Hour'] = pd.DatetimeIndex(fut_without_df['Time_UTC']).hour

# Merge the datasets together based on common BA, year, month, day, and hour combinations:
analysis_df = pd.merge(hist_with_df, fut_with_df, how='left', on=['BA', 'Month', 'Day', 'Hour'])
analysis_df = analysis_df[['BA', 'Year_x', 'Month', 'Day', 'Hour', 'historic_with_pop', 'future_with_pop']].copy()
analysis_df = pd.merge(analysis_df, hist_without_df, how='left', on=['BA', 'Month', 'Day', 'Hour'])
analysis_df = analysis_df[['BA', 'Year_x', 'Month', 'Day', 'Hour', 'historic_with_pop', 'future_with_pop', 'historic_without_pop']].copy()
analysis_df = pd.merge(analysis_df, fut_without_df, how='left', on=['BA', 'Month', 'Day', 'Hour'])
analysis_df = analysis_df[['BA', 'Year_x', 'Month', 'Day', 'Hour', 'historic_with_pop', 'future_with_pop', 'historic_without_pop', 'future_without_pop']].copy()

# Convert the time to a datetime variable:
analysis_df['date'] = analysis_df[['Year_x', 'Month', 'Day', 'Hour']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
analysis_df['Time_UTC'] = pd.to_datetime(analysis_df['date'], format='%Y-%m-%d-%H')

# Compute the normalized loads:
analysis_df['norm_with_pop'] = 100 * (analysis_df['future_with_pop'].div(analysis_df['historic_with_pop']) - 1)
analysis_df['norm_without_pop'] = 100 * (analysis_df['future_without_pop'].div(analysis_df['historic_without_pop']) - 1)

# Only keep the columns that are needed:
analysis_df = analysis_df[['BA', 'Time_UTC', 
                           'historic_with_pop', 'future_with_pop', 'norm_with_pop',
                           'historic_without_pop', 'future_without_pop', 'norm_without_pop']].copy()

# Return the dataframe
analysis_df


In [ ]:
# Make a list of all of the BAs in the "analysis_df":
bas = analysis_df['BA'].unique()

# Create an empty dataframe and store the results:
ba_df = pd.DataFrame()
    
# Loop over the BAs and compute their mean effects:
for i in range(len(bas)):
    # Subset to just the data for the BA being processed:
    subset_df = analysis_df[analysis_df['BA'].isin([bas[i]])].copy()
    
    # Compute the date of the peak historical load:
    peak_day = subset_df['Time_UTC'].loc[subset_df['historic_with_pop'].idxmax()] 
    min_date = pd.to_datetime(peak_day) - timedelta(days=3.5)
    max_date = pd.to_datetime(peak_day) + timedelta(days=3.5)
    
    # Subset to only the week around the historical peak load value:
    peak_df = subset_df.loc[(subset_df['Time_UTC'] >= min_date) & (subset_df['Time_UTC'] < max_date)]
    
    # Output the mean difference:
    ba_df.loc[i, 'BA'] = bas[i]
    ba_df.loc[i, 'Mean_With_Pop'] = subset_df['norm_with_pop'].mean().round(2)
    ba_df.loc[i, 'Max_With_Pop'] = subset_df['norm_with_pop'].max().round(2)
    ba_df.loc[i, 'Peak_With_Pop'] = peak_df['norm_with_pop'].mean().round(2)
    ba_df.loc[i, 'Mean_Without_Pop'] = subset_df['norm_without_pop'].mean().round(2)
    ba_df.loc[i, 'Max_Without_Pop'] = subset_df['norm_without_pop'].max().round(2)
    ba_df.loc[i, 'Peak_Without_Pop'] = peak_df['norm_without_pop'].mean().round(2)

# Subset the dataframe to only BAs in the WECC:
ba_df = ba_df[ba_df['BA'].isin(['AVA','AZPS','BANC','BPAT','CHPD','CISO','DEAA','DOPD','EPE',
                                'GCPD','IID','IPCO','LDWP','NEVP','NWMT','PACE','PACW','PGE',
                                'PNM','PSCO','PSEI','SCL','SRP','TEPC','TIDC','TPWR','WACM',
                                'WALC','WAUW'])]
    
# Return the dataframe:
ba_df


In [ ]:
# Create an x-axis the length of the dataframe to be used in plotting:
x_axis = np.arange(len(ba_df))

# Make the plot:
plt.figure(figsize=(25, 20))
plt.subplot(221)
plt.bar(x_axis, ba_df.sort_values(by=['Mean_With_Pop'], ascending=True)['Mean_With_Pop'], 0.75)
plt.xticks(x_axis, ba_df.sort_values(by=['Mean_With_Pop'], ascending=True)['BA'], rotation=90)
plt.grid()
plt.xlabel('Balancing Authority')
plt.ylabel('Mean Load Change in 2052 Relative to 2012 [%]')
plt.title('Mean Effect with Climate and Socioeconomic Impacts: ' + scenario_to_analze)

plt.subplot(222)
plt.bar(x_axis, ba_df.sort_values(by=['Mean_Without_Pop'], ascending=True)['Mean_Without_Pop'], 0.75)
plt.xticks(x_axis, ba_df.sort_values(by=['Mean_Without_Pop'], ascending=True)['BA'], rotation=90)
plt.grid()
plt.xlabel('Balancing Authority')
plt.ylabel('Mean Load Change in 2052 Relative to 2012 [%]')
plt.title('Mean Effect with Climate Only Impacts: ' + scenario_to_analze)

plt.subplot(223)
plt.bar(x_axis, ba_df.sort_values(by=['Peak_With_Pop'], ascending=True)['Peak_With_Pop'], 0.75)
plt.xticks(x_axis, ba_df.sort_values(by=['Peak_With_Pop'], ascending=True)['BA'], rotation=90)
plt.grid()
plt.xlabel('Balancing Authority')
plt.ylabel('Mean Load Change in 2052 Relative to 2012 [%]')
plt.title('Peak Week Effect with Climate and Socioeconomic Impacts')

plt.subplot(224)
plt.bar(x_axis, ba_df.sort_values(by=['Peak_Without_Pop'], ascending=True)['Peak_Without_Pop'], 0.75)
plt.xticks(x_axis, ba_df.sort_values(by=['Peak_Without_Pop'], ascending=True)['BA'], rotation=90)
plt.grid()
plt.xlabel('Balancing Authority')
plt.ylabel('Mean Load Change in 2052 Relative to 2012 [%]')
plt.title('Peak Week Effect with Climate Only Impacts')

plt.subplots_adjust(wspace=0.15, hspace=0.4)

filename = ('BA_Comparison_2052_' + scenario_to_analze + '.png')

plt.savefig(os.path.join(image_output_dir, filename), dpi=300, bbox_inches='tight', facecolor='white')
